In [1]:
import pandas as pd

In [2]:
tracking_data = pd.read_csv('data/tracking.csv')

In [3]:
tracking_data

,id_half,t,id_actor,id_team,x,y
0,1,0,-1,-1,5261,3383
1,1,0,396315,1884426,10408,3406
2,1,0,401281,1935290,3426,1870
3,1,0,411367,1935290,5234,5158
4,1,0,429386,1884426,5545,5823
...,...,...,...,...,...,...
345018,1,600000,339987,1935290,3518,1681
345019,1,600000,405843,1935290,4715,4244
345020,1,600000,398681,1935290,3884,823
345021,1,600000,289964,1935290,2411,1834


In [4]:
events_data = pd.read_csv('data/events.csv')
events_data

,event_id,half_time,time,player_id,team_id,event
0,0,1,625.68,358112.0,1935290.0,Kick Off
1,1,1,625.68,358112.0,1935290.0,Pass
2,2,1,626.69,339987.0,1935290.0,Reception
3,3,1,627.61,339987.0,1935290.0,Pass
4,4,1,628.38,398681.0,1935290.0,Reception
...,...,...,...,...,...,...
257,257,1,1195.66,429386.0,1884426.0,Attempt at Goal
258,258,1,1198.79,NaN,NaN,Ball Out of Play
259,259,1,1221.29,270948.0,1935290.0,Throw in
260,260,1,1223.17,358112.0,1935290.0,Pass


### 1. Join both datasets such that each event is associated with the correct coordinates of the corresponding player at the given time. The units for time, x, and y can be chosen as you like

Milliseconds to seconds

In [5]:
tracking_data['time'] = tracking_data['t']/1000 
tracking_data

,id_half,t,id_actor,id_team,x,y,time
0,1,0,-1,-1,5261,3383,0.0
1,1,0,396315,1884426,10408,3406,0.0
2,1,0,401281,1935290,3426,1870,0.0
3,1,0,411367,1935290,5234,5158,0.0
4,1,0,429386,1884426,5545,5823,0.0
...,...,...,...,...,...,...,...
345018,1,600000,339987,1935290,3518,1681,600.0
345019,1,600000,405843,1935290,4715,4244,600.0
345020,1,600000,398681,1935290,3884,823,600.0
345021,1,600000,289964,1935290,2411,1834,600.0


Synchronisation of time variables from KickOff

In [6]:
init_time = events_data.loc[events_data['event'] == 'Kick Off']['time'][0]
init_time

625.68

In [7]:
events_data['reset_time'] = events_data['time'] - init_time
events_data

,event_id,half_time,time,player_id,team_id,event,reset_time
0,0,1,625.68,358112.0,1935290.0,Kick Off,0.00
1,1,1,625.68,358112.0,1935290.0,Pass,0.00
2,2,1,626.69,339987.0,1935290.0,Reception,1.01
3,3,1,627.61,339987.0,1935290.0,Pass,1.93
4,4,1,628.38,398681.0,1935290.0,Reception,2.70
...,...,...,...,...,...,...,...
257,257,1,1195.66,429386.0,1884426.0,Attempt at Goal,569.98
258,258,1,1198.79,NaN,NaN,Ball Out of Play,573.11
259,259,1,1221.29,270948.0,1935290.0,Throw in,595.61
260,260,1,1223.17,358112.0,1935290.0,Pass,597.49


Join DataFrames.

The column 'half_time' has been used to avoid conflicts between the extra time of the first part and the second part, even if the dataset does not contain this data.

In [8]:
join = pd.merge(events_data, tracking_data, how='left', left_on=['reset_time','half_time','player_id'], right_on=['time','id_half','id_actor'])

In [9]:
result = join[['event_id','half_time', 'reset_time', 'player_id', 'team_id', 'event', 'x', 'y']]
result

,event_id,half_time,reset_time,player_id,team_id,event,x,y
0,0,1,0.00,358112.0,1935290.0,Kick Off,5267.0,3374.0
1,1,1,0.00,358112.0,1935290.0,Pass,5267.0,3374.0
2,2,1,1.01,339987.0,1935290.0,Reception,NaN,NaN
3,3,1,1.93,339987.0,1935290.0,Pass,NaN,NaN
4,4,1,2.70,398681.0,1935290.0,Reception,NaN,NaN
...,...,...,...,...,...,...,...,...
257,257,1,569.98,429386.0,1884426.0,Attempt at Goal,NaN,NaN
258,258,1,573.11,NaN,NaN,Ball Out of Play,NaN,NaN
259,259,1,595.61,270948.0,1935290.0,Throw in,NaN,NaN
260,260,1,597.49,358112.0,1935290.0,Pass,NaN,NaN


### 2. Calculate the length of the ball trajectory from the initial kickoff to the first "Ball Out of Play" event. The unit should be in meters.

In [10]:
ball = tracking_data.loc[(tracking_data['id_actor'] == -1)]
ball

,id_half,t,id_actor,id_team,x,y,time
0,1,0,-1,-1,5261,3383,0.00
23,1,40,-1,-1,5218,3404,0.04
46,1,80,-1,-1,5175,3426,0.08
69,1,120,-1,-1,5130,3447,0.12
92,1,160,-1,-1,5087,3467,0.16
...,...,...,...,...,...,...,...
344908,1,599840,-1,-1,5742,879,599.84
344931,1,599880,-1,-1,5775,897,599.88
344954,1,599920,-1,-1,5806,914,599.92
344977,1,599960,-1,-1,5833,925,599.96


First Ball Out of Play

In [11]:
grouped = events_data.groupby(events_data.event)

out_of_play = grouped.get_group("Ball Out of Play")
out_of_play

,event_id,half_time,time,player_id,team_id,event,reset_time
12,12,1,643.34,NaN,NaN,Ball Out of Play,17.66
64,64,1,736.57,NaN,NaN,Ball Out of Play,110.89
83,83,1,779.76,NaN,NaN,Ball Out of Play,154.08
98,98,1,816.24,NaN,NaN,Ball Out of Play,190.56
120,120,1,854.60,NaN,NaN,Ball Out of Play,228.92
124,124,1,906.97,NaN,NaN,Ball Out of Play,281.29
175,175,1,1008.74,NaN,NaN,Ball Out of Play,383.06
189,189,1,1038.29,NaN,NaN,Ball Out of Play,412.61
198,198,1,1067.18,NaN,NaN,Ball Out of Play,441.50
209,209,1,1094.75,NaN,NaN,Ball Out of Play,469.07


In [12]:
first_ball_out_of_play_time = out_of_play.iloc[0]['reset_time']
first_ball_out_of_play_time

17.660000000000082

In [13]:
ball_trajectory = ball[ball['time'] < first_ball_out_of_play_time]
ball_trajectory

,id_half,t,id_actor,id_team,x,y,time
0,1,0,-1,-1,5261,3383,0.00
23,1,40,-1,-1,5218,3404,0.04
46,1,80,-1,-1,5175,3426,0.08
69,1,120,-1,-1,5130,3447,0.12
92,1,160,-1,-1,5087,3467,0.16
...,...,...,...,...,...,...,...
10051,1,17480,-1,-1,8453,4730,17.48
10074,1,17520,-1,-1,8497,4746,17.52
10097,1,17560,-1,-1,8540,4760,17.56
10120,1,17600,-1,-1,8568,4760,17.60


In [14]:
import math

In [15]:
total_distance = 0

last_x = ball_trajectory.iloc[0]['x']
last_y = ball_trajectory.iloc[0]['y']
        
for row in ball_trajectory.itertuples():

    x = row[5]
    y = row[6]

    distance = math.dist([last_x, last_y], [x, y])
    total_distance += distance
    
    last_x = x
    last_y = y
    
total_distance

12397.439289443539

cm to meters

In [16]:
total_distance/100

123.97439289443538

### 3. Add a new column to the first table that flags for each pass and cross event if the pass was successful or misplaced.

Verification that the next event is from the same team

In [17]:
status = []

for index, row in events_data.iterrows():
    if row['event'] == 'Pass' or row['event'] == 'Cross':
        if row['team_id'] == events_data.iloc[index + 1]['team_id']:
            status.append(1)
        else:
            status.append(0)
    else:
        status.append(None)

events_data['status'] = status

In [18]:
events_data

,event_id,half_time,time,player_id,team_id,event,reset_time,status
0,0,1,625.68,358112.0,1935290.0,Kick Off,0.00,NaN
1,1,1,625.68,358112.0,1935290.0,Pass,0.00,1.0
2,2,1,626.69,339987.0,1935290.0,Reception,1.01,NaN
3,3,1,627.61,339987.0,1935290.0,Pass,1.93,1.0
4,4,1,628.38,398681.0,1935290.0,Reception,2.70,NaN
...,...,...,...,...,...,...,...,...
257,257,1,1195.66,429386.0,1884426.0,Attempt at Goal,569.98,NaN
258,258,1,1198.79,NaN,NaN,Ball Out of Play,573.11,NaN
259,259,1,1221.29,270948.0,1935290.0,Throw in,595.61,NaN
260,260,1,1223.17,358112.0,1935290.0,Pass,597.49,1.0


### 4. Which player had the most passes, which had the best pass completion rate in percent?

In [19]:
passes = events_data[(events_data["event"]=='Pass') | (events_data["event"]=='Cross')]
passes

,event_id,half_time,time,player_id,team_id,event,reset_time,status
1,1,1,625.68,358112.0,1935290.0,Pass,0.00,1.0
3,3,1,627.61,339987.0,1935290.0,Pass,1.93,1.0
5,5,1,628.38,398681.0,1935290.0,Pass,2.70,1.0
7,7,1,629.96,289964.0,1935290.0,Pass,4.28,1.0
9,9,1,632.59,379955.0,1935290.0,Pass,6.91,1.0
...,...,...,...,...,...,...,...,...
247,247,1,1164.76,270948.0,1935290.0,Pass,539.08,0.0
249,249,1,1173.62,429383.0,1884426.0,Pass,547.94,0.0
251,251,1,1174.95,398681.0,1935290.0,Pass,549.27,0.0
254,254,1,1182.36,396315.0,1884426.0,Pass,556.68,0.0


In [20]:
player_passes = passes.groupby(['player_id']).size().reset_index(name='total_passes').sort_values('total_passes', ascending=False)
player_passes.iloc[0]

player_id       289964.0
total_passes        20.0
Name: 1, dtype: float64

In [21]:
accurate_passes = passes[passes["status"]==1].groupby(['player_id']).size().reset_index(name='accurate_passes').sort_values('accurate_passes', ascending=False)
accurate_passes

,player_id,accurate_passes
1,289964.0,19
7,379955.0,12
3,339987.0,8
12,401281.0,8
10,396315.0,4
19,439538.0,4
11,398681.0,4
9,396056.0,3
0,270948.0,3
8,395433.0,3


In [22]:
passes_ratio = pd.merge(player_passes, accurate_passes, left_on='player_id', right_on='player_id')
passes_ratio['ratio'] = passes_ratio['accurate_passes'] * 100 / passes_ratio['total_passes']

In [23]:
passes_ratio

,player_id,total_passes,accurate_passes,ratio
0,289964.0,20,19,95.000000
1,379955.0,12,12,100.000000
2,339987.0,9,8,88.888889
3,401281.0,8,8,100.000000
4,439538.0,6,4,66.666667
5,396315.0,6,4,66.666667
6,270948.0,5,3,60.000000
7,398681.0,5,4,80.000000
8,396056.0,4,3,75.000000
9,405843.0,4,2,50.000000


In [24]:
passes_ratio.iloc[passes_ratio['ratio'].idxmax()]

player_id          379955.0
total_passes           12.0
accurate_passes        12.0
ratio                 100.0
Name: 1, dtype: float64